### Libraries 

In [1]:
#initial step
from skimage import data, io, filters, feature, measure, transform, morphology
from skimage.color import rgb2gray
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os

### Constants

In [2]:
MAX_ANLGE=160
MIN_ANGLE=20

MAX_AREA=0
MIN_AREA=0


### Utilities

In [3]:
# Show the figures / plots inside the notebook
# instead of io.imshow(gaussedImg) and plt.show() each time
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show() 

#calculate the distance between two points
def CalcDist(p1,p2):
    p1=np.copy(p1)
    p2=np.copy(p2)
    dist=np.sqrt((((p1-p2)**2).sum()))
    return dist

#calculate the perimeter of a contour
def CalcPerimeter(contour):
    ret=0
    prev=contour[-1]
    for cur in contour:
        ret+=CalcDist(cur,prev)
        prev=cur
    return ret

#caculate angle of three points using law of cosines
#arccos((P12^2 + P13^2 - P23^2) / (2 * P12 * P13))
def CalcAngle(points):
    p12=CalcDist(points[0],points[1])
    p13=CalcDist(points[2],points[1])
    p23=CalcDist(points[0],points[2])

    angle=np.arccos((p12**2+p13**2-p23**2)/(2*p12*p13))
    return angle*180/np.arccos(-1)

#caculate Area a polygon using Shoelace formula
def CalcArea(x,y):
    area=0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))
    return area



### Preprocessing Step

In [4]:
#TO DO: try applying the preprocessing step on 4 different part of the pic
def preprocessingStep(img):

    grayImg=rgb2gray(img)
    
    if(np.max(grayImg) < 1.01):
        grayImg=grayImg*255
        
    gaussedImg=filters.gaussian(grayImg)

    addedConst=0
    thresh=filters.threshold_otsu(grayImg)+addedConst
    threImg=np.copy(gaussedImg)
    threImg[threImg>thresh]=255
    threImg[threImg<=thresh]=0
    
    return threImg

### Find and Filter Contours

In [5]:
#check if the contour is likely to be a card
def CheckContour(contour):
    
    if(len(contour)!=5):
        return False
    
    area=CalcArea(contour[:4,1],contour[:4,0])
    if(area > MAX_AREA or area < MIN_AREA):
        return False
    
    points=np.copy(contour)
    points=np.vstack([points, points[1]])    
    
    for i in range(4):
        angle=CalcAngle(points[i:i+3])
        if(angle >MAX_ANLGE or angle < MIN_ANGLE):
            return False
    
    return True

#get candidate cards
def findCardsStep(threImg):
    
    contours=measure.find_contours(threImg)

    cards=[]
    for contour in contours:
        perimeter=CalcPerimeter(contour)
        approx=measure.approximate_polygon(contour,.08*perimeter)
        if(CheckContour(approx)==True):
            cards.append(approx)
            
    return cards

### Sort Card Corners

In [6]:
#sort the corners of the card
def SortCorners(approxContour):
    
    corners=np.copy(approxContour[:4])
    
    maxIndx=0
    maxDis=0
    
    for i in range(4):
        dis=CalcDist(corners[i],corners[(i+1)%4])
        if(dis > maxDis):
            maxIndx=i
            maxDis=dis
       
    shift=0
    if(CalcDist(corners[maxIndx],corners[(maxIndx+1)%4]) <  CalcDist(corners[(maxIndx+1)%4],corners[(maxIndx+2)%4])):
        shift=maxIndx
    else:
        shift=maxIndx+1
        
    corners=np.roll(corners,-shift,axis=0)
    
    return corners

### Perspective Step

In [7]:
row=350
col=250

def perspectiveStep(cards,img):

    transCards=[]
    
    for card in cards:
        dst=(SortCorners(card))[:,[1,0]].astype(int)
        src=np.array([
            [0,0],
            [col,0],
            [col,row],
            [0,row]
        ])
        transMatrix=transform.ProjectiveTransform()
        transMatrix.estimate(src,dst)
        warpedImg=transform.warp(img,transMatrix,output_shape=(row,col))
        transCards.append(warpedImg)
        
    return transCards

### Cut and Threshold the Top Left Corner

In [8]:
#get the rank,suit

CORNER_HIEGHT=90
CORNER_WIDTH=40
CORNER_THRESH=60

def GetCorner(transImg):

    corner=transImg[10:CORNER_HIEGHT,10:CORNER_WIDTH]

    grayCorner=rgb2gray(corner)
    
    if(np.max(grayCorner) < 1.01):
        grayCorner=grayCorner*255  
    
    thresh=filters.threshold_otsu(grayCorner)
    
    threCorner=grayCorner.copy()
    threCorner[threCorner>=thresh]=255
    threCorner[threCorner<thresh]=0
    
    return threCorner
    

### Integrate and Get Corners

In [9]:
def applyAlgo(img):
    threImg=preprocessingStep(img)
    cards=findCardsStep(threImg)
    transCards=perspectiveStep(cards,img)

    ret=[]
    for transCard in transCards:
       ret.append(GetCorner(transCard))
    
    return ret
    
    

### Get the Largest Contour In The Corner -> which is the wanted symbol

In [10]:
RANK_WIDTH=30
RANK_HEIGHT=40

#get largest contour
def LargestContour(img):
    
    contours=measure.find_contours(img)

    maxArea=0
    ret=[]
    for contour in contours:
        area=CalcArea(contour[:,1],contour[:,0])
        if(area>maxArea):
            maxArea=area
            ret=contour
    
    if(len(ret)==0):
        img=125
        return img

    l=round(np.min(ret[:,1]))
    r=round(np.max(ret[:,1]))
    t=round(np.min(ret[:,0]))
    b=round(np.max(ret[:,0]))

    img=img[t:b,l:r]
    img = cv2.resize(img, (RANK_WIDTH,RANK_HEIGHT), 0, 0)

    return img


In [14]:
#apply the algorithm on the dataset


dire="D:/programing/faculty of engineering/4_third year in cmp/first term/image processing/project/sevens/"

count=0
for filename in os.listdir(dire):
    img=io.imread(dire+filename)

    h=img.shape[0]
    w=img.shape[1]

    MAX_AREA=0.314*w*h
    MIN_AREA=0.014*w*h

    rets=applyAlgo(img)
    # show_images([img])
    for ret in rets:
        SUIT=40
        RANK=50
        suitImg=ret[SUIT:,:]
        rankImg=ret[:RANK,:]

        suitImg=LargestContour(suitImg)
        rankImg=LargestContour(rankImg)

        # cv2.imwrite('D:/programing/faculty of engineering/4_third year in cmp/first term/image processing/project/output/'+str(count)+'.jpg', suitImg)
        # count=count+1
        cv2.imwrite('D:/programing/faculty of engineering/4_third year in cmp/first term/image processing/project/output/'+str(count)+'.jpg', rankImg)
        # show_images([rankImg])
        count=count+1